In [1]:
import pandas as pd
import numpy as np
import pprint as pp
import re, string
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import praw

import credentials as creds

In [2]:
# https://praw.readthedocs.io/en/latest/getting_started/
# https://www.reddit.com/dev/api/

In [3]:
CLIENT_ID = creds.client_id()
CLIENT_SECRET_KEY = creds.client_secret_key()


r = praw.Reddit(client_id = CLIENT_ID,
                client_secret = CLIENT_SECRET_KEY,
                user_agent = 'RedditorMatch')

print(r.read_only)

True


In [4]:
def getThreads(subreddit, limit = 20):
    thread_ids = []
    
    hotThreads = subreddit.hot(limit = limit)
    time.sleep(2)
    topThreads = subreddit.top(limit = limit)
    time.sleep(2)
    newThreads = subreddit.new(limit = limit)    
    
    for t in hotThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
        
    for t in topThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
        
    for t in newThreads:
        if (t.id not in thread_ids):
            thread_ids.append(t.id)
    
    return(thread_ids)

In [5]:
def getUsersFromSubreddit(subredditIds):
    if(len(subredditIds) < 1):
        return None
    
    usernames = []
    
    ## this process takes a while...    

    for thread_id in subredditIds:        
        thread = r.submission(id = thread_id)
        time.sleep(2)
        
        try:
            username = thread.author.name # gets the author of the thread 
            if (username is not None and username not in usernames):
                print("--- Adding user:", username)
                usernames.append(username)
        except:
            print("------ Unable to get thread author. Moving on...")      

        # get all authors of the comments in the thread
        allComments = thread.comments.list()
        for c in allComments:
            try:
                username = str(c.author)
                if (username is not None and username not in usernames):
                    print("--- Adding user:", username)
                    usernames.append(username)
            except:
                print("------ failed. moving on...")
                
        time.sleep(2)
                
    return(usernames)      

In [6]:
def cleanText(text):
    text = text.replace(',', ' ').replace(':', ' ')\
                .replace('...', ' ')\
                .replace('?', ' ')\
                .replace('!', ' ')\
                .replace(';', ' ')\
                .replace('\n', ' ').replace('\r', '') # replace newlines and page breaks     
                
    text = re.sub(r'([^\s\w]|_)+', '', text) # remove non-alphanumeric characters but leave the spaces
    text = re.sub(' +',' ', text) # remove double spaces        
    return(text)

In [7]:
def getRedditor(username):
    try:     
        username = username.strip()
        user = r.redditor(username)
    except Exception as e: 
        print(e)
        return None
    
    return(user)

In [8]:
def getUserComments(username, commentsLimit = 1000):   
    userComments = []
    

    counter = 0
    while (counter <= 3):
        user = getRedditor(username)
        if (user is not None):
            break
        else:
            counter = counter + 1               
            print("------------ Retry #", counter, "for user:", username)
            time.sleep(15)
        
    if (user is None):
        print("------ ERROR: unable to retrieve user info:", username)
        return None  
    
    time.sleep(2)
    
    try:
        newComments = user.comments.new(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve new comments. moving on...")
        newComments = None
    
    time.sleep(2)
    
    try:
        hotComments = user.comments.hot(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve hot comments. moving on...")
        hotComments = None
    
    time.sleep(2)
    
    try:
        controversialComments = user.comments.controversial(limit = commentsLimit)
    except Exception as e: 
        print(e)
        print("------ ERROR: unable to retrieve controversial comments. moving on...")
        controversialComments = None

    # new comments    
    try:
        if (newComments is not None):
            for c in newComments:
                comment = cleanText(c.body)        
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")
    
    # hot comments
    try:
        if (hotComments is not None):
            for c in hotComments:  
                comment = cleanText(c.body)        
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")

    # controversial comments
    try:
        if (controversialComments is not None):
            for c in controversialComments:
                comment = cleanText(c.body)     
                if (comment not in userComments):
                    userComments.append(comment)
    except:
        print("Failed...moving on...")
        
    print("------ Retrieved", len(userComments), "comments for:", username)
    return(userComments)

In [9]:
def subExists(subreddit):
    exists = True
    try:
        r.subreddits.search_by_name(subreddit, exact = True)
    except:
        exists = False
    return(exists)

In [10]:
def scrapeCommentsFromSubreddit(subreddit):
    
    UserArray = []
    CommentArray = []  
    
    exists = subExists(subreddit)
    
    if (exists == False):
        print("ERROR: subreddit '", subreddit, "' does not exist...")
        return None
    
    sub = r.subreddit(subreddit)
    
    threads = getThreads(sub)
    
    if (len(threads) < 1):
        print("ERROR: unable to retrieve threads for subreddit:", subreddit)
    else:
        print("Retrieved ", len(threads), "threads...")
    
    users = getUsersFromSubreddit(threads)
    
    if (len(users) < 1):
        print("ERROR: unable to retrieve users for subreddit:", subreddit)
    else:
        print("Retrieved ", len(users), "users...")
    
    for user in users:
        try:
            userComments = getUserComments(user)

            if (userComments is not None):
                userComments = ".".join(userComments)
                UserArray.append(user)
                CommentArray.append(userComments)
        except:
            print("ERROR: something failed badly. moving on...")
        
    df = pd.DataFrame(UserArray, columns = ["Username"])
    df["Comments"] = CommentArray
    
    print("Outputting dataframe with ", len(df), "records")
    
    return(df)

In [11]:
#mizzou = scrapeCommentsFromSubreddit("mizzou")

#mizzou.to_csv("data/mizzou.csv", index = False)

In [ ]:
umich = scrapeCommentsFromSubreddit("uofm")

umich.to_csv("data/umich.csv", index = False)

Retrieved  43 threads...
--- Adding user: dragpent
--- Adding user: yes_no_yes_yes_yes
--- Adding user: LoveMockdown
--- Adding user: vishnureddy17
--- Adding user: Asdamaje
--- Adding user: Asianhead
--- Adding user: gafavid
--- Adding user: Bryggyth
--- Adding user: AChillWolverine
--- Adding user: Antwohlf
--- Adding user: joefigs
--- Adding user: smelborp_for_preside
--- Adding user: Brandon1027
--- Adding user: KitchenEnd
--- Adding user: zelTram
--- Adding user: CrosscutJester8
--- Adding user: None
--- Adding user: OhOkayThanks
--- Adding user: Madigan37
--- Adding user: GawdGo
--- Adding user: evodie
--- Adding user: LoHershal
--- Adding user: stemich
--- Adding user: throwawayinthefire
--- Adding user: newunivmich
--- Adding user: michigan2020
--- Adding user: trillic
--- Adding user: okl_617
--- Adding user: soraaxle
--- Adding user: Ripped_Atheist
--- Adding user: DarthUnnamed
--- Adding user: ultimamax
--- Adding user: omylime
--- Adding user: im2slick4u
--- Adding user: qu

--- Adding user: skinschamps2000
--- Adding user: SignorSarcasm
--- Adding user: jefforjo
--- Adding user: yoshirox7
--- Adding user: Halsey117
--- Adding user: Kalium
--- Adding user: Bill_of_Wrongs
--- Adding user: EpiceEmilie
--- Adding user: msando
--- Adding user: awinnie
--- Adding user: battman820
--- Adding user: Trosso
--- Adding user: for_ever_a_lone
--- Adding user: Roboticide
--- Adding user: CraigularB
--- Adding user: beah8er
--- Adding user: yanchovilla
--- Adding user: gr3yhill
--- Adding user: emsyy61
--- Adding user: elh93
--- Adding user: Matt_KB
--- Adding user: MaizeRage48
--- Adding user: Gold_Jacobson
--- Adding user: staffinfection92
--- Adding user: coachfortner
--- Adding user: wangj
--- Adding user: Ftwpkerz
--- Adding user: Insert_Whiskey
--- Adding user: cheeburglar
--- Adding user: zerodivider
--- Adding user: kirkt
--- Adding user: hydrohydro
--- Adding user: DrBlotto
--- Adding user: normalguy821
--- Adding user: GenitalFurbies
--- Adding user: Sat_Nav66

------ Retrieved 486 comments for: colinmhayes2
------ Retrieved 1 comments for: SpaceYooper
------ Retrieved 667 comments for: santoast_
------ Retrieved 3 comments for: a_op
------ Retrieved 141 comments for: inkqi
------ Retrieved 27 comments for: jon-sn0w
------ Retrieved 861 comments for: tbh11
------ Retrieved 125 comments for: zaser77
------ Retrieved 20 comments for: juandaddy
------ Retrieved 100 comments for: thedc96
------ Retrieved 1268 comments for: msattam
------ Retrieved 61 comments for: orked_
------ Retrieved 130 comments for: rosemary515
------ Retrieved 675 comments for: heedlessly3
------ Retrieved 1754 comments for: fracta1
------ Retrieved 51 comments for: one_soup_snake
------ Retrieved 296 comments for: Selbeven
------ Retrieved 1131 comments for: MWolverine63
------ Retrieved 271 comments for: Dropping_Dimes
------ Retrieved 793 comments for: sensenomake
------ Retrieved 708 comments for: lazyfirefly
------ Retrieved 537 comments for: Claassy
------ Retrieved 

------ Retrieved 194 comments for: goatsofsyria
------ Retrieved 464 comments for: hail_to_the_victors
------ Retrieved 1528 comments for: arghabargh
------ Retrieved 1921 comments for: 1337Gandalf
------ Retrieved 957 comments for: Rowbond
------ Retrieved 154 comments for: bluewildcat
------ Retrieved 1372 comments for: skinschamps2000
------ Retrieved 1808 comments for: SignorSarcasm
------ Retrieved 49 comments for: jefforjo
------ Retrieved 92 comments for: yoshirox7
------ Retrieved 840 comments for: Halsey117
------ Retrieved 1872 comments for: Kalium
------ Retrieved 385 comments for: Bill_of_Wrongs
------ Retrieved 1395 comments for: EpiceEmilie
------ Retrieved 372 comments for: msando
------ Retrieved 1825 comments for: awinnie
------ Retrieved 213 comments for: battman820
------ Retrieved 1942 comments for: Trosso
------ Retrieved 98 comments for: for_ever_a_lone
------ Retrieved 1987 comments for: Roboticide
------ Retrieved 1872 comments for: CraigularB
------ Retrieved 9

In [ ]:
chicago = scrapeCommentsFromSubreddit("chicago")

chicago.to_csv("data/chicago.csv", index = False)

Retrieved  48 threads...
--- Adding user: AutoModerator
--- Adding user: alyse0688
--- Adding user: gene66
--- Adding user: kittenlover27
--- Adding user: sonictech170
--- Adding user: chisox100
--- Adding user: unburntt
--- Adding user: Thirtytw031
--- Adding user: PlaneZebra
--- Adding user: justagirlintheworld
--- Adding user: bobphilip
--- Adding user: TropangTexter
--- Adding user: vjosiah
--- Adding user: MrSassybaskets
--- Adding user: sleepysol
--- Adding user: drunkenbarbecueninja
--- Adding user: TyGO28
--- Adding user: whatsgucci13
--- Adding user: Phaedrus813
--- Adding user: theCovertoit
--- Adding user: oncogenie
--- Adding user: alyanumbers
--- Adding user: 2wheels_1pup
--- Adding user: tomtom1234121231
--- Adding user: sdhardy89
--- Adding user: hippagun
--- Adding user: theworldbystorm
--- Adding user: bradatlarge
--- Adding user: effing-what
--- Adding user: Notstrongbad
--- Adding user: GordonBombaye
--- Adding user: trippin113
--- Adding user: ocshawn
--- Adding use

--- Adding user: ooo-ooo-oooyea
--- Adding user: Never_enough_hummus
--- Adding user: Areia
--- Adding user: MrKFCGravy
--- Adding user: TheIceCreamMansBro2
--- Adding user: ChiBarneski
--- Adding user: koavf
--- Adding user: FirePowerCR
--- Adding user: badzachlv01
--- Adding user: ZeOppositeOfProgress
--- Adding user: lovesStrawberryCake
--- Adding user: suprememetrocard
--- Adding user: WalterSweetness
--- Adding user: thereisaway
--- Adding user: Cyzzacle1
--- Adding user: urbanplanner
--- Adding user: OrangeinDorne
--- Adding user: seppo420gringo
--- Adding user: Woodolude
--- Adding user: breweryshitpantsguy
--- Adding user: SalukiKnightX
--- Adding user: rcrobot
--- Adding user: pooteeweet28
--- Adding user: forcejump
--- Adding user: Eurynom0s
--- Adding user: lunchlady55
--- Adding user: LeZygo
--- Adding user: gray52064
--- Adding user: ejhickey
--- Adding user: Dystopiq
--- Adding user: spade_andarcher
--- Adding user: ChemPeddler
--- Adding user: stepcut251
--- Adding user:

--- Adding user: BigBen2010
--- Adding user: BBana
--- Adding user: Schwannson
--- Adding user: lpython
--- Adding user: kennious
--- Adding user: Jollygrand
--- Adding user: BunnySamurai7
--- Adding user: shiverstar
--- Adding user: nnotserPx
--- Adding user: Greenmagic847
--- Adding user: lordrefa
--- Adding user: sarcasticspastic
--- Adding user: LambChops1909
--- Adding user: UIUC13
--- Adding user: IAmOfficial
--- Adding user: jlarrison
--- Adding user: mr_krink1e
--- Adding user: clumsybravado
--- Adding user: MaroonDogX
--- Adding user: Dennis7d
--- Adding user: unsavoryginger
--- Adding user: farfle10
--- Adding user: fightinscot
--- Adding user: ReallyRick
--- Adding user: designgoddess
--- Adding user: aj191
--- Adding user: RobotJoe
--- Adding user: Pretzeloid
--- Adding user: R_TOKAR
--- Adding user: Pyrobolser
--- Adding user: dmw1987
--- Adding user: CasualEcon
--- Adding user: chi-reply
--- Adding user: Tills_Monocle
--- Adding user: Danither
--- Adding user: Pantherophi

--- Adding user: Pikajane
--- Adding user: AreolaSanchez
--- Adding user: Allferik
--- Adding user: TanikaTubman
--- Adding user: MagnusPI
--- Adding user: yoshkow
--- Adding user: DisturbedAle
--- Adding user: cantpickusername
--- Adding user: schaef51
--- Adding user: themightykobold
--- Adding user: Schmoopster
--- Adding user: makeswordcloudsagain
--- Adding user: andymorphic
--- Adding user: chizmanzini
--- Adding user: burdundi
--- Adding user: PostPostModernism
--- Adding user: Im_A_Racehorse
--- Adding user: crapidrawatwork
--- Adding user: good1god
--- Adding user: bbiggs32
--- Adding user: chinacat1977
--- Adding user: paladin10025
--- Adding user: surrender_cobra
--- Adding user: vir4030
--- Adding user: lexrp
--- Adding user: raosion
--- Adding user: Originatek
--- Adding user: krazykarol123
--- Adding user: BarroomBard
--- Adding user: AtticusWarhol
--- Adding user: steev227
--- Adding user: Bob8435
--- Adding user: MENOTMEMAYBE
--- Adding user: elshgi
--- Adding user: Iri

--- Adding user: brunchusevenmx
--- Adding user: desidude52
--- Adding user: iamkingoftheworld
--- Adding user: I_Have_A_Girls_Name
--- Adding user: homescribe_
--- Adding user: RecklessBacon
--- Adding user: shingox
--- Adding user: D-DOGGIE
--- Adding user: thirdcoaster
--- Adding user: Daier_Mune
--- Adding user: jcpianiste
--- Adding user: samainteasy
--- Adding user: TacticalCat
--- Adding user: Ayepocalypse
--- Adding user: JeanHelicase
--- Adding user: sirredthebig
--- Adding user: LightCascade
--- Adding user: ethan1231
--- Adding user: FitK3vin
--- Adding user: Strong__Belwas
--- Adding user: notquiteotaku
--- Adding user: duckies_wild
--- Adding user: AnotherPint
--- Adding user: KazarakOfKar
--- Adding user: Up-The-Butt_Jesus
--- Adding user: Flick1981
--- Adding user: ChicagoShadow
--- Adding user: bleepbloopdederp
--- Adding user: AdmiralAkbar1
--- Adding user: HippieTrippie
--- Adding user: Kevin-W
--- Adding user: bigdaddyEm
--- Adding user: ChronicallyPale
--- Adding us

...

------ Retrieved 1016 comments for: CoachPlatitude
------ Retrieved 1755 comments for: PossessedToSkate
Failed...moving on...
Failed...moving on...
Failed...moving on...
------ Retrieved 0 comments for: nerfAvari
------ Retrieved 1247 comments for: mcman7890
------ Retrieved 1792 comments for: -crave
------ Retrieved 1950 comments for: iforgot120
------ Retrieved 1826 comments for: ConsummateK
------ Retrieved 1773 comments for: schwibbity
------ Retrieved 144 comments for: KindOfLingers
------ Retrieved 802 comments for: yooper-pete
------ Retrieved 1869 comments for: weeglos
------ Retrieved 866 comments for: Latch
------ Retrieved 13 comments for: randomgirl74
------ Retrieved 525 comments for: dogsdontfly
------ Retrieved 307 comments for: j3AoLBsU
------ Retrieved 1359 comments for: Procyonid
------ Retrieved 1659 comments for: IAmRasputin
------ Retrieved 1424 comments for: well_shore
------ Retrieved 186 comments for: KOA04
------ Retrieved 263 comments for: jallenclark
------ R

------ Retrieved 222 comments for: truckmonth
------ Retrieved 679 comments for: UtterFaillure
------ Retrieved 1430 comments for: CT4Heisman
------ Retrieved 1538 comments for: PyrollisAhFiros
------ Retrieved 1950 comments for: cyantist
------ Retrieved 1905 comments for: ive_lost_my_keys
------ Retrieved 1779 comments for: RomanCavalry
------ Retrieved 1854 comments for: fugged_up_shib
------ Retrieved 1699 comments for: MashdPotatoJohnson
------ Retrieved 356 comments for: HaveFunDying
------ Retrieved 1743 comments for: Chicago-Rican
------ Retrieved 153 comments for: lotsalasagna
------ Retrieved 1727 comments for: Stormray117
------ Retrieved 181 comments for: Pablo_Louserama
------ Retrieved 223 comments for: Bear_Cat16
------ Retrieved 1253 comments for: Blue-Jasmine
------ Retrieved 1244 comments for: seabass86
------ Retrieved 1824 comments for: St1ng
------ Retrieved 1899 comments for: markycapone
Failed...moving on...
Failed...moving on...
Failed...moving on...
------ Retr

------ Retrieved 1178 comments for: c_hand
------ Retrieved 1961 comments for: Blizzaldo
------ Retrieved 1832 comments for: Kojakle
------ Retrieved 1630 comments for: CptSpaulding
------ Retrieved 1619 comments for: justindaniel
------ Retrieved 1638 comments for: CLOUD_STRIFE27
Failed...moving on...
Failed...moving on...
Failed...moving on...
------ Retrieved 0 comments for: NewThoughtsForANewMe
------ Retrieved 1536 comments for: SonOfSlam
------ Retrieved 1846 comments for: stanleypup
------ Retrieved 1451 comments for: Trbluedemon
------ Retrieved 1323 comments for: eskimoboob
------ Retrieved 1584 comments for: eprada
------ Retrieved 1764 comments for: NeverDieKris
------ Retrieved 1069 comments for: Galzreon
------ Retrieved 1278 comments for: EdGein_Green
------ Retrieved 1737 comments for: Janky_Pants
------ Retrieved 463 comments for: 4343528
------ Retrieved 41 comments for: skorpiankh
------ Retrieved 1455 comments for: Silvedl
------ Retrieved 337 comments for: carnstar


------ Retrieved 176 comments for: svengoolies
------ Retrieved 78 comments for: wmartin123
------ Retrieved 1550 comments for: Ellis_Dee-25
------ Retrieved 1853 comments for: NamelessNamek
------ Retrieved 345 comments for: DamenAve
------ Retrieved 1795 comments for: Erosis
------ Retrieved 1732 comments for: the_trout
------ Retrieved 1549 comments for: TheJessKiddin
------ Retrieved 1937 comments for: imguralbumbot
------ Retrieved 103 comments for: sandwich_influence
------ Retrieved 886 comments for: theStraightUp
------ Retrieved 1708 comments for: grendel_x86
------ Retrieved 981 comments for: brokentyro
------ Retrieved 1781 comments for: obelus
------ Retrieved 1937 comments for: SpyCake1
------ Retrieved 1808 comments for: ProfessorPedro
------ Retrieved 1091 comments for: CordeAmare
------ Retrieved 1800 comments for: IamARealEstateBroker
------ Retrieved 1282 comments for: IshyMoose
------ Retrieved 1494 comments for: Agost1jm
------ Retrieved 1811 comments for: the_stars

------ Retrieved 1786 comments for: Halgrind
------ Retrieved 1797 comments for: majorgeneralporter
------ Retrieved 1271 comments for: Undersleep
------ Retrieved 1165 comments for: JOEY2X
------ Retrieved 88 comments for: Maschining
------ Retrieved 1820 comments for: TheHoneyBadger23
------ Retrieved 80 comments for: Roberson12
------ Retrieved 1167 comments for: Jfjfjdjdjj
------ Retrieved 158 comments for: Ltholland
------ Retrieved 1940 comments for: bettorworse
------ Retrieved 1309 comments for: HIGHHAMMER
------ Retrieved 381 comments for: seventwenty3
------ Retrieved 319 comments for: thechitowncubs
------ Retrieved 1678 comments for: CrunchyBones
------ Retrieved 1609 comments for: wescoe23
------ Retrieved 15 comments for: shameonlyonedied
------ Retrieved 1930 comments for: ZeMoose
------ Retrieved 1774 comments for: tatanka_truck
------ Retrieved 1058 comments for: WS6Grumbles
------ Retrieved 8 comments for: 9000saabturbo
------ Retrieved 128 comments for: Bobsagit14
--

In [ ]:
columbia = scrapeCommentsFromSubreddit("columbiamo")

columbia.to_csv("data/columbia.csv", index = False)

In [ ]:
dems = scrapeCommentsFromSubreddit("democrats")

dems.to_csv("data/dems.csv", index = False)

In [ ]:
reps = scrapeCommentsFromSubreddit("republicans")

reps.to_csv("data/reps.csv", index = False)

In [ ]:
bball = scrapeCommentsFromSubreddit("CollegeBasketball")

bball.to_csv("data/bball.csv", index = False)

In [ ]:
austin = scrapeCommentsFromSubreddit("austin")

austin.to_csv("data/austin.csv", index = False)